# propósito da análise

## importando bibliotecas e o arquivo

In [ ]:
import pandas as pd
import numpy as np
import re

In [ ]:
df=pd.read_csv("../../shark_project/shark_attack/attacks.csv", encoding='latin1')

## explorando

In [ ]:
df

In [ ]:
df.shape

In [ ]:
df.columns

In [ ]:
df.info()

In [ ]:
df.isnull().sum()

In [ ]:
# O que as últimas colunas contem?
df[['Case Number.1', 'Case Number.2', 'original order', 'Unnamed: 22', 'Unnamed: 23']].head()

In [ ]:
# essas colunas não contém informações relevantes para a minha análise e podem ser removidas

In [ ]:
# Existem linhas duplicadas?
print('Linhas duplicadas:' )
print(df.duplicated().sum())

# Limpeza

## Remover duplicatas

In [ ]:
shark_attack = df.drop_duplicates()
print('Número de duplicatas removidas: ', len(df) - len(shark_attack))

In [ ]:
shark_attack.shape

## Remover colunas irrelevantes

In [ ]:
shark_attack = shark_attack.drop(['Case Number', 'Date','Location','Name','Sex ','Age','Time', 'Species ','Investigator or Source', 'pdf', 'href formula', 'href','Case Number.1', 'Case Number.2', 'original order', 'Unnamed: 22', 'Unnamed: 23'], axis =1)

In [ ]:
shark_attack.columns

In [ ]:
shark_attack.loc[:, "Country":"Activity"] = shark_attack.loc[:, "Country":"Activity"].transform(lambda series: series.str.strip().str.title())

In [ ]:
shark_attack.head(10)

In [ ]:
shark_attack = shark_attack.rename(columns={'Fatal (Y/N)': 'Fatal'})

In [ ]:
shark_attack.info()

In [ ]:
shark_attack.isnull().sum()

## Limpeza da coluna Year

Quero considerar os ataques que aconteceram nos últimos 50 anos

In [ ]:
shark_attack['Year'].unique()

In [ ]:
shark_attack.Year.isnull().sum()

In [ ]:
shark_attack = shark_attack.dropna(subset=['Year'])

In [ ]:
shark_attack.Year.isnull().sum()

In [ ]:
shark_attack.Year = shark_attack.Year.astype(np.int64)
shark_attack['Year'].unique()

In [ ]:
shark_attack = shark_attack[shark_attack['Year'] > 1967] 

shark_attack

## Coluna Fatal

In [ ]:
shark_attack["Fatal"].unique()

In [ ]:
shark_attack.Fatal.isnull().sum()

In [ ]:
shark_attack = shark_attack.fillna('No Data')

In [ ]:
shark_attack['Fatal']=shark_attack['Fatal'].replace({'UNKNOWN':'No Data', 'Unknown':'No Data', 'M':'N', '2017':'No Data'})
shark_attack['Fatal'].unique()

In [ ]:
shark_attack['Fatal'].value_counts()

## coluna País

In [ ]:
shark_attack.Country.unique()

In [ ]:
shark_attack.Country.value_counts().head(20)

## Coluna Area

In [ ]:
shark_attack['Area'].value_counts().head(30)

## Tipo
Types of attack
The GSAF categorizes scavenging bites on humans as "questionable incidents."

- PROVOKED
Provoked attacks occur when a human touches, hooks, nets, or otherwise aggravates the animal. Incidents that occur outside of a shark's natural habitat, such as aquariums and research holding-pens, are considered provoked, as are all incidents involving captured sharks. Sometimes humans inadvertently provoke an attack, such as when a surfer accidentally hits a shark with a surf board.

- UNPROVOKED
Hit-and-run attack
Sneak Attack
Bump-and-bite attack
For more information on how to differentiate PROVOKED vs UNPROVOKED attacks : https://en.wikipedia.org/wiki/Shark_attack#Types_of_attacks

TYPE OF ATTACK (Considering Injuries)
I wanted to use the information on the Injury column to define if the human hooked, shot, or provoked the shark. However, after mapping the Injury column, I noticed that the ammount of Provoked attacks count from this mapping technique varied very slightly from the number of provoked attacks on the Type column. A decission was made to disregard these two additional cases.
Size of the shark according to Species column

In [ ]:
shark_attack.Type.value_counts()

In [ ]:
shark_attack.Type = shark_attack.Type.replace({'Boating' : 'Boat', 'Boatomg': 'Boat', 'Invalid':'No Data', 'Questionable':'No Data'})
shark_attack.Type.value_counts()

## Coluna Activity

In [ ]:
shark_attack["Activity"].unique()

In [ ]:
shark_attack.Activity.value_counts().head(30)

In [ ]:
shark_attack.Activity.isnull().sum()

In [ ]:
#testes

#shark_attack2 = shark_attack["Activity"].str.replace('.*([Dd]iv).*',"Diving")
#shark_attack2 = shark_attack["Activity"].str.replace('.*([Pp]addl).*',"Paddling")
#shark_attack2 = shark_attack["Activity"].str.replace('.*([Ss]wim).*',"Swimming")
#shark_attack2 = shark_attack["Activity"].str.replace('.*([Ff]ish).*',"Fishing")
#shark_attack2 = shark_attack["Activity"].str.replace('.*([Ss]urf).*',"Surfing")


In [ ]:
most_attacked_activity = shark_attack

In [ ]:
# Boat, Boating and Boatomg to 1 category
category = lambda x: 'Surfing' if 'Surfing' in x or 'Windsurfing' in x or 'Surf' in x or 'Body Boarding' in x or 'Boogie Boarding' in x else x
most_attacked_activity.Activity = shark_attack.Activity.map(category)

# Boat, Boating and Boatomg to 1 category
category = lambda x: 'Swimming' if 'Swimming' in x  or 'Swim' in x or 'Snorkeling' in x or 'Floating' in x else x
most_attacked_activity.Activity = shark_attack.Activity.map(category)

# Boat, Boating and Boatomg to 1 category
category = lambda x: 'Wading' if 'Wading' in x or 'Standing' in x else x
most_attacked_activity.Activity = shark_attack.Activity.map(category)

# Boat, Boating and Boatomg to 1 category
category = lambda x: 'Diving' if 'Diving' in x or 'Spearfishing' in x else x
most_attacked_activity.Activity = shark_attack.Activity.map(category)

# Boat, Boating and Boatomg to 1 category
category = lambda x: 'Fishing' if 'Fishing' in x else x
most_attacked_activity.Activity = shark_attack.Activity.map(category)

# Boat, Boating and Boatomg to 1 category
category = lambda x: 'Boat' if 'Boat' in x or 'Paddling' in x or 'Paddle' in x or 'Kayaking' in x else x
most_attacked_activity.Activity = shark_attack.Activity.map(category)


most_attacked_activity.Activity.value_counts().head(20)

In [ ]:
# Separar as atividades por Categorias

#Surfing
shark['Activity_Category'] = shark.Activity.map(lambda x: 'Surfing' if 'Surfing' in x or 'Windsurfing' in x or 'Surf' in x 
                                               or 'Body Boarding' in x or 'Boogie Boarding' in x else x)

shark['Activity_Category'] = shark.Activity.map(lambda x: 'Swimming' if 'Swimming' in x  or 'Snorkeling' in x else x)
#shark['Activity_Category'] = shark.Activity.map(category2)




#Surfing
#category1 = lambda x: 'Surfing' if 'Surfing' in x or 'Windsurfing' in x or 'Surf' in x or 'Body Boarding' in x or 'Boogie Boarding' in x else x
#shark['Activity_Category'] = shark.Activity.map(category1)
# = shark.Activity.map(category1

# Swimming
#category2 = lambda x: 'Swimming' if 'Swimming' in x  or 'Snorkeling' in x else x
#shark['Activity_Category'] = shark.Activity.map(category2)

# Wading
#category = lambda x: 'Wading' if 'Wading' in x or 'Standing' in x or 'Walking' else x
#shark['Activity_Category'] = shark.Activity.map(category2)

# Diving
#category = lambda x: 'Diving' if 'Diving' in x or 'Spearfishing' in x else x
#shark_attack.Activity = shark_attack.Activity.map(category)

# Fishing
#category = lambda x: 'Fishing' if 'Fishing' in x else x
#shark_attack.Activity = shark_attack.Activity.map(category)

# Boating
#category = lambda x: 'Boat' if 'Boat' in x or 'Paddling' in x or 'Paddle' in x or 'Kayaking' in x or 'Boating' in x else x
#shark_attack.Activity = shark_attack.Activity.map(category)

#shark_attack.Activity.value_counts().head(20)

In [ ]:
shark.Activity_Category.value_counts().head(15)


In [ ]:
# Swimming
category2 = lambda x: 'Swimming' if 'Swimming' in x  or 'Snorkeling' in x else x
shark['Activity_Category'] = shark.Activity.map(category2)

In [ ]:
shark.Activity_Category.value_counts().head(15)

In [ ]:
shark.head()

In [ ]:
shark_attack = shark_attack[['Year','Type','Country','Area','Activity','Activity_Category','Injury','Fatal']]
shark_attack.head()

In [ ]:
# substituir valores nulos por 'NOT SPECIFIED' pois as informações mais relevantes que é ANO e ATIVIDADE ja estão limpas

In [ ]:
shark_attack.isnull().sum()

In [ ]:
shark_attack = shark_attack.fillna('NOT SPECIFIED')

In [ ]:
shark_attack.isnull().sum()